


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [ ]:
# Load the data and preprocess data and store corpus in raw_text
import nltk
import nltk.data
nltk.download('punkt')
from nltk.tokenize import word_tokenize

raw_text = nltk.data.load('corpus.txt', format='raw').decode('utf-8')

# converting _very_ to very
def preprocessor(match):
  return re.sub(r'[_]', "", match.group())

pattern = re.compile(r'[_]*[^\s]+[_]+|[_]+[^\s]+[_]*')
raw_text = pattern.sub(preprocessor, raw_text)
raw_text = re.sub(r'[^\w\s]', '', raw_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tokens = []
for sentences in re.split("\n", raw_text):
    curr_tokens = word_tokenize(sentences, language='english')
    tokens.extend([token.lower() for token in curr_tokens if token.isalnum()]) 

print(f'The number of tokens = {len(tokens)}')
print(f'The number of unique tokens = {len(set(tokens))}')

The number of tokens = 26381
The number of unique tokens = 2751


In [ ]:
# Hyperparameters of the model
vocab_size = 2754 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [ ]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, lower=True, oov_token=oov_tok)
tokenizer.fit_on_texts([raw_text])
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'the': 2, 'and': 3, 'to': 4, 'a': 5, 'she': 6, 'it': 7, 'of': 8, 'said': 9, 'i': 10, 'alice': 11, 'in': 12, 'you': 13, 'was': 14, 'that': 15, 'as': 16, 'her': 17, 'at': 18, 'on': 19, 'with': 20, 'all': 21, 'had': 22, 'but': 23, 'for': 24, 'so': 25, 'be': 26, 'very': 27, 'not': 28, 'what': 29, 'this': 30, 'little': 31, 'they': 32, 'he': 33, 'out': 34, 'its': 35, 'is': 36, 'down': 37, 'one': 38, 'up': 39, 'his': 40, 'about': 41, 'if': 42, 'then': 43, 'no': 44, 'were': 45, 'like': 46, 'know': 47, 'them': 48, 'would': 49, 'went': 50, 'herself': 51, 'again': 52, 'do': 53, 'have': 54, 'when': 55, 'could': 56, 'or': 57, 'there': 58, 'thought': 59, 'off': 60, 'time': 61, 'me': 62, 'queen': 63, 'into': 64, 'see': 65, 'how': 66, 'did': 67, 'your': 68, 'who': 69, 'king': 70, 'well': 71, 'dont': 72, 'began': 73, 'my': 74, 'an': 75, 'im': 76, 'now': 77, 'by': 78, 'mock': 79, 'turtle': 80, 'quite': 81, 'hatter': 82, 'gryphon': 83, 'think': 84, 'are': 85, 'way': 86, 'their': 87, 'much': 

In [ ]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([raw_text])[0]
print(tokens)

[299, 10, 37, 2, 827, 11, 14, 275, 4, 101, 27, 466, 8, 342, 78, 17, 414, 19, 2, 1056, 3, 8, 343, 129, 4, 53, 137, 57, 583, 6, 22, 828, 64, 2, 467, 17, 414, 14, 829, 23, 7, 22, 44, 683, 57, 1057, 12, 7, 3, 29, 36, 2, 209, 8, 5, 467, 59, 11, 169, 683, 57, 1057, 25, 6, 14, 830, 12, 17, 344, 345, 16, 71, 16, 6, 56, 24, 2, 584, 170, 145, 17, 415, 27, 585, 3, 528, 321, 2, 1058, 8, 416, 5, 1472, 49, 26, 684, 2, 529, 8, 187, 39, 3, 1059, 2, 1473, 55, 276, 5, 146, 107, 20, 1474, 159, 228, 277, 78, 17, 58, 14, 129, 25, 27, 1060, 12, 15, 831, 67, 11, 84, 7, 25, 27, 88, 34, 8, 2, 86, 4, 248, 2, 107, 91, 4, 249, 122, 150, 122, 150, 10, 171, 26, 530, 55, 6, 59, 7, 112, 1061, 7, 1062, 4, 17, 15, 6, 250, 4, 54, 1475, 18, 30, 23, 18, 2, 61, 7, 21, 160, 81, 685, 23, 55, 2, 107, 1476, 175, 5, 417, 34, 8, 35, 1063, 3, 102, 18, 7, 3, 43, 322, 19, 11, 1064, 4, 17, 202, 24, 7, 1477, 586, 17, 345, 15, 6, 22, 103, 126, 251, 5, 107, 20, 346, 5, 1063, 57, 5, 417, 4, 191, 34, 8, 7, 3, 1478, 20, 587, 6, 228, 586, 

In [ ]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  26329


In [ ]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
print(y)
y = np_utils.to_categorical(dataY)
print(y.shape)

[  36    2  209 ... 1451  792    2]
(26329, 2754)


In [ ]:
from keras.callbacks import ModelCheckpoint

# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

# saving the best model based on validation loss
mcp_save = ModelCheckpoint('word_lstm.hdf5', save_best_only=True, monitor='val_loss', mode='min')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 100)           275400    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 2754)              355266    
Total params: 715,146
Trainable params: 715,146
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=50, batch_size=128, validation_split=0.2, callbacks=mcp_save, validation_freq=1) 

Epoch 1/50
165/165 [==============================] - 4s 21ms/step - loss: 6.4372 - accuracy: 0.0544 - val_loss: 6.2512 - val_accuracy: 0.0849
Epoch 2/50
165/165 [==============================] - 3s 17ms/step - loss: 5.9594 - accuracy: 0.0560 - val_loss: 6.1831 - val_accuracy: 0.0849
Epoch 3/50
165/165 [==============================] - 3s 16ms/step - loss: 5.8385 - accuracy: 0.0590 - val_loss: 6.1917 - val_accuracy: 0.0900
Epoch 4/50
165/165 [==============================] - 3s 16ms/step - loss: 5.7349 - accuracy: 0.0671 - val_loss: 6.1810 - val_accuracy: 0.0894
Epoch 5/50
165/165 [==============================] - 3s 17ms/step - loss: 5.6456 - accuracy: 0.0701 - val_loss: 6.1712 - val_accuracy: 0.0934
Epoch 6/50
165/165 [==============================] - 3s 17ms/step - loss: 5.5567 - accuracy: 0.0770 - val_loss: 6.1651 - val_accuracy: 0.1003
Epoch 7/50
165/165 [==============================] - 3s 16ms/step - loss: 5.4541 - accuracy: 0.0902 - val_loss: 6.1349 - val_accuracy: 0.1033

In [ ]:
model.load_weights('word_lstm.hdf5')

In [ ]:
# list out keys and values separately 
key_list = list(word_index.keys()) 
val_list = list(word_index.values()) 

reverse_word_map = dict()
for i in range(len(val_list)):
  reverse_word_map[val_list[i]] = key_list[i]

In [ ]:
import heapq

def fix_p(prediction):
  number = 1
  while (sum(prediction)>1.0):
    number = 10+number
    prediction[heapq.nsmallest(number, range(len(prediction)), prediction.take)] = 0
  return prediction.tolist()

In [ ]:
# Complete the code to return next n words greedily
def next_tokens(input_str, n): 
	print ("Seed: \n", input_str)
	
	tokens = tokenizer.texts_to_sequences([input_str])
	final_tokens = []
	for i in range(n):
		prediction = model.predict(tokens, verbose=0)

		pred = fix_p(prediction[0])
		next_token = np.argmax(np.random.multinomial(1, pred))

		del tokens[0][0]
		tokens[0].append(next_token)
		final_tokens.append(next_token)

	return input_str + ' ' + ' '.join([reverse_word_map[value+1] for value in final_tokens])

In [ ]:
# pick a random seed
seed = 10
np.random.seed(seed)

start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ' '.join([reverse_word_map[value] for value in pattern])

print(next_tokens(input_str, 10))

Seed: 
 not so she went in search of her hedgehog the hedgehog was engaged in a fight with another hedgehog which seemed to alice an excellent opportunity for croqueting one of them with the other the only difficulty was that her flamingo was gone across to the other side of the
not so she went in search of her hedgehog the hedgehog was engaged in a fight with another hedgehog which seemed to alice an excellent opportunity for croqueting one of them with the other the only difficulty was that her flamingo was gone across to the other side of the rosetree and just as she tried one and no sort


In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
input_str = ' '.join([reverse_word_map[value] for value in tokenizer.texts_to_sequences([input_str])[0]])
print(next_tokens(input_str , 10))

Seed: 
 the boy laughed at the fright he had caused this time the <OOV> left angrily the <OOV> day as the boy went up the small <OOV> he suddenly saw a <OOV> <OOV> his <OOV> he cried as hard as he could <OOV> <OOV> <OOV> <OOV> but not a <OOV> <OOV> came to help him the <OOV> thought that he was trying to <OOV> them again and did not come to <OOV> him or his <OOV>
the boy laughed at the fright he had caused this time the <OOV> left angrily the <OOV> day as the boy went up the small <OOV> he suddenly saw a <OOV> <OOV> his <OOV> he cried as hard as he could <OOV> <OOV> <OOV> <OOV> but not a <OOV> <OOV> came to help him the <OOV> thought that he was trying to <OOV> them again and did not come to <OOV> him or his <OOV> certainly said the duchess thats a little golden key and


# Character based LSTM Model 1

In [ ]:
# User the preprocess data and create raw_text
raw_text = nltk.data.load('corpus.txt', format='raw').decode('utf-8')

# converting _very_ to very
def preprocessor(match):
  return re.sub(r'[_]', "", match.group())

pattern = re.compile(r'[_]*[^\s]+[_]+|[_]+[^\s]+[_]*')
raw_text = pattern.sub(preprocessor, raw_text)
raw_text = re.sub(r'[^\w\s,.]', '', raw_text)
# create mapping of unique characters to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [ ]:
# Print the total characters and character vacob size
n_chars = len(raw_text)
n_vocab = len(chars)
print(n_vocab)

57


In [ ]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  137356


In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [ ]:
embedding_dim =100
max_length =100

In [ ]:
model2 = Sequential()
model2.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model2.add(LSTM(256))
model2.add(Dropout(0.2))
model2.add(Dense(y.shape[1], activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

# saving the best model based on validation loss
checkpoint_filepath = 'char1-lstm.hdf5'
mcp_save = ModelCheckpoint(checkpoint_filepath, save_best_only=True, monitor='val_loss', mode='min')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          5700      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 57)                14649     
Total params: 385,917
Trainable params: 385,917
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.fit(X, y, epochs=20, batch_size=128, validation_split=0.2, callbacks=mcp_save, validation_freq=1)
# model.load_weights(checkpoint_filepath)

Epoch 1/20
859/859 [==============================] - 18s 21ms/step - loss: 2.3342 - accuracy: 0.3468 - val_loss: 1.9851 - val_accuracy: 0.4346
Epoch 2/20
859/859 [==============================] - 17s 20ms/step - loss: 1.8461 - accuracy: 0.4656 - val_loss: 1.7543 - val_accuracy: 0.4944
Epoch 3/20
859/859 [==============================] - 18s 20ms/step - loss: 1.6508 - accuracy: 0.5152 - val_loss: 1.6244 - val_accuracy: 0.5290
Epoch 4/20
859/859 [==============================] - 18s 20ms/step - loss: 1.5234 - accuracy: 0.5474 - val_loss: 1.5362 - val_accuracy: 0.5525
Epoch 5/20
859/859 [==============================] - 17s 20ms/step - loss: 1.4335 - accuracy: 0.5718 - val_loss: 1.4845 - val_accuracy: 0.5634
Epoch 6/20
859/859 [==============================] - 18s 20ms/step - loss: 1.3638 - accuracy: 0.5897 - val_loss: 1.4539 - val_accuracy: 0.5747
Epoch 7/20
859/859 [==============================] - 18s 20ms/step - loss: 1.3048 - accuracy: 0.6054 - val_loss: 1.4373 - val_accuracy:

In [ ]:
#implement mapping of integer to character
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(int_to_char)

{0: '\n', 1: ' ', 2: ',', 3: '.', 4: 'A', 5: 'B', 6: 'C', 7: 'D', 8: 'E', 9: 'F', 10: 'G', 11: 'H', 12: 'I', 13: 'J', 14: 'K', 15: 'L', 16: 'M', 17: 'N', 18: 'O', 19: 'P', 20: 'Q', 21: 'R', 22: 'S', 23: 'T', 24: 'U', 25: 'V', 26: 'W', 27: 'X', 28: 'Y', 29: 'Z', 30: 'a', 31: 'b', 32: 'c', 33: 'd', 34: 'e', 35: 'f', 36: 'g', 37: 'h', 38: 'i', 39: 'j', 40: 'k', 41: 'l', 42: 'm', 43: 'n', 44: 'o', 45: 'p', 46: 'q', 47: 'r', 48: 's', 49: 't', 50: 'u', 51: 'v', 52: 'w', 53: 'x', 54: 'y', 55: 'z', 56: 'ù'}


In [ ]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	final_pattern = [value for value in pattern]
	for i in range(x):

		char_tokens = np.reshape(pattern, (1, len(pattern), 1))
		prediction = model2.predict(char_tokens, verbose=0)
		
		pred = fix_p(prediction[0])
		index = np.argmax(np.random.multinomial(1, pred))

		pattern.pop(0)
		pattern.append(index)
		final_pattern.append(index)

	return ''.join([int_to_char[value] for value in final_pattern])

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(f'The original string:\n {input_str}')
print(f'The predicted string:\n {predict_next_100_chars(pattern,100)}')

The original string:
 to usurpation and conquest.
Edwin and Morcar, the earls of Mercia and Northumbria Ugh said the Lory,
The predicted string:
 to usurpation and conquest.
Edwin and Morcar, the earls of Mercia and Northumbria Ugh said the Lory, remaimed other of muchness deep that it was a large little double in far of this sort of the garden


In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 100 characeters from given input_str as input and generate next 200 characters.
pattern = []
for char in input_str:
  pattern.append(char_to_int[char])
  if len(pattern) == 100:
    break

print(''.join([int_to_char[value] for value in pattern]))
print(predict_next_100_chars(pattern,200))

The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, a
The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, and the Hatter and the Queen jumped augh on for down, she fellow, the Mock Turtle recade, to herself that the cooks of the way better, I think she said to the chimney.
Were the chimney ons stop in the 


## Character based LSTM Model 2


In [ ]:
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# saving the best model based on validation loss
checkpoint_filepath = 'char2-lstm.hdf5'
mcp_save = ModelCheckpoint(checkpoint_filepath, save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
model1.fit(X, y, epochs=20, batch_size=64, validation_split=0.2, callbacks=mcp_save, validation_freq=1)
# model1.load_weights(checkpoint_filepath)

Epoch 1/20
1717/1717 [==============================] - 44s 26ms/step - loss: 2.2518 - accuracy: 0.3670 - val_loss: 1.8467 - val_accuracy: 0.4680
Epoch 2/20
1717/1717 [==============================] - 43s 25ms/step - loss: 1.6933 - accuracy: 0.5063 - val_loss: 1.6041 - val_accuracy: 0.5335
Epoch 3/20
1717/1717 [==============================] - 43s 25ms/step - loss: 1.5054 - accuracy: 0.5514 - val_loss: 1.5065 - val_accuracy: 0.5557
Epoch 4/20
1717/1717 [==============================] - 43s 25ms/step - loss: 1.3925 - accuracy: 0.5808 - val_loss: 1.4510 - val_accuracy: 0.5744
Epoch 5/20
1717/1717 [==============================] - 43s 25ms/step - loss: 1.3154 - accuracy: 0.6016 - val_loss: 1.4200 - val_accuracy: 0.5840
Epoch 6/20
1717/1717 [==============================] - 43s 25ms/step - loss: 1.2545 - accuracy: 0.6163 - val_loss: 1.4132 - val_accuracy: 0.5836
Epoch 7/20
1717/1717 [==============================] - 43s 25ms/step - loss: 1.2021 - accuracy: 0.6310 - val_loss: 1.3943 -

In [ ]:
# Generate the sequence similar to above methods
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	final_pattern = [value for value in pattern]
	for i in range(x):
		char_tokens = np.reshape(pattern, (1, len(pattern), 1))
		prediction = model1.predict(char_tokens, verbose=0)
		
		pred = fix_p(prediction[0])
		index = np.argmax(np.random.multinomial(1, pred))

		pattern.pop(0)
		pattern.append(index)
		final_pattern.append(index)

	return ''.join([int_to_char[value] for value in final_pattern])

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(input_str)
print(predict_next_100_chars(pattern,200))

o on for some time without interrupting it.
They were learning to draw, the Dormouse went on, yawnin
o on for some time without interrupting it.
They were learning to draw, the Dormouse went on, yawning near the way in a minute, and no which such a sudden lear, but it is a bright idea came there was exactly and the amfrassand. Alice went been with 
ne we cas, and he could be cats but Alice could no


In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 100 characeters from given input_str as input and generate next 200 characters.
pattern = []
for char in input_str:
  pattern.append(char_to_int[char])
  if len(pattern) == 100:
    break

print(predict_next_100_chars(pattern,200))

The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, and felt quite roline and with her both bring about in her last began and quite anxiously in a pressance was nothing out enough back again, cried Alice, a sippica was very much was.
As it must be a Mar


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:**
The words/characters generated by these models try to mimic the training data and is sometimes succesful in doing so. Thus better sentences were generated in case of training data in some instances when compared to test data. But even in training data, due to exposure bias they fail sometimes to generate grammatically correct sentences.  



**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:** The sequence of characters generated by the second model seem to be better than first model in general. The reasons for this seem to be the larger model size. For example: Model 1 generates the word: 'remaimed' which is not part of training data but words generated by Model 2 are all valid. Apart from this both of them generate sequence of words which don't follow grammar.





# Bonus (Not to be graded)

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [ ]:
!pip install transformers

In [ ]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [ ]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [ ]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)